In [27]:
from PIL import Image, ImageDraw
import numpy
import shutil
import os
import json
from shapely.geometry import Polygon
import shapely.wkt


In [28]:
image_source = 'data/train_images/pre_images/'
dest = 'data/train_images/pre_buildings/'

pre_hurricane_images = sorted([f for f in os.listdir(image_source)])

# source = 'data/train_images/pre_images/hurricane-florence_00000012_pre_disaster.png'

# image = Image.open(source).convert("RGBA")
#image

In [29]:
def getBuildingImage(image, poly_xy, building_poly):
    imArray = numpy.asarray(image)
    
    # create mask
    maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
    ImageDraw.Draw(maskIm).polygon(poly_xy, outline=1, fill=1)
    mask = numpy.array(maskIm)
    
    # assemble new image (uint8: 0-255)
    newImArray = numpy.empty(imArray.shape,dtype='uint8')

    # colors (three first columns, RGB)
    newImArray[:,:,:3] = imArray[:,:,:3]

    # transparency (4th column)
    newImArray[:,:,3] = mask*255

    # back to Image from numpy
    newIm = Image.fromarray(newImArray, "RGBA")
    
    #crop out just the building from image
    poly_bounds = building_poly.bounds
    cropped_building_img = newIm.crop(poly_bounds)
    
    #put center of cropped image onto standarized image size
    img_w, img_h = cropped_building_img.size
    background = Image.new('RGBA', (100, 75), (255, 255, 255, 0))
    bg_w, bg_h = background.size
    offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
    background.paste(cropped_building_img, offset)
    return background
    
    
    

In [36]:
## get json data for this file
json_source = 'data/train_labels/pre_labels/'
pre_hurricane_jsons = sorted([f for f in os.listdir(json_source)])

## cuts out building polygons from each image, standardrizes format, and saves images to data folder
#len(pre_hurricane_images)
for image_index in range(1):
    image = Image.open(image_source + pre_hurricane_images[image_index]).convert("RGBA")
    with open (json_source + pre_hurricane_jsons[image_index]) as f:
        json_data = json.load(f)
    building_counter = 0
    for building in json_data['features']['xy']:
        building_wkt = building['wkt']
        building_poly = shapely.wkt.loads(building_wkt)
        x, y = building_poly.exterior.coords.xy
        poly_xy = [(x[i], y[i]) for i in range(len(x))]
        formatted_building_image = getBuildingImage(image, poly_xy, building_poly)
        formatted_building_image.save(dest + pre_hurricane_images[image_index].replace('.png', '') + '_building_' + str(building_counter) + '.png')
        building_counter += 1
        

